In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import field
import spatial
import timesteppers
import scipy.sparse.linalg as spla
from scipy import sparse

In [ ]:
def plot_2D_field(xm, ym, data):
    fig = plt.figure(figsize=(4,3))
    ax = fig.add_subplot(111)
    pcm = ax.pcolormesh(xm, ym, data)
    ax.set_aspect(1)
    fig.colorbar(pcm)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

## Introduction

I've made some changes to our files so we can solve equations in multiple spatial dimensions. Now you can make multiple `Grid`s and put them together into a `Domain`. Previously, we would often pass the `Grid` to different objects (`Field`, `Operator`, etc.); now instead we will pass the `Domain`.

When you make derivative operators, you now need to specify the direction the derivative is being taken in.

## Advection Problem

Now we're going to do a simple advection problem. We will solve
$$\partial_t c + \boldsymbol{u}\boldsymbol{\cdot}\boldsymbol{\nabla} c = 0$$
Here $\boldsymbol{u}=(u, v)$ is a constant advection velocity. We will solve this problem using an explicit method.

In [ ]:
class Advection:
    
    def __init__(self, X, u, v, dcdx, dcdy):
        pass

In [ ]:
N = 100
grid_x = field.UniformPeriodicGrid(N, 2*np.pi)
grid_y = field.UniformPeriodicGrid(N, 2*np.pi)
domain = field.Domain((grid_x, grid_y))
x, y = domain.values()
xm, ym = domain.plotting_arrays()

To timestep the problem, we use a new `ExplicitTimestepper`.

## Implicit Timestepping

Implicit timestepping is a bit more complicated in 2D. We need to think about the matrix form of our operators.

In [ ]:
N = 10
grid_x = field.UniformPeriodicGrid(N, 2*np.pi)
grid_y = field.UniformPeriodicGrid(N, 2*np.pi)
domain = field.Domain((grid_x, grid_y))
x, y = domain.values()

u = field.Field(domain)

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
pcm = ax.pcolormesh(xm, ym, c.data)
ax.set_aspect(1)
fig.colorbar(pcm)
ax.set_xlabel('x')
ax.set_ylabel('y')
fig.canvas.draw()
while ts.t < 2*np.pi:
    ts.step(dt)
    if ts.iter % 50 == 0:
        pcm.set_array(np.ravel(c.data))
        fig.canvas.draw()

## Diffusion Problem

As an example of a problem we can solve with implicit timestepping, we will solve the 2D diffusion problem
$$\partial_t c - D \nabla^2 c = \partial_t c - D (\partial_x^2 + \partial_y^2) c = 0.$$
This equation has the azimuthally symmetric solution
$$c(r,t) = \frac{1}{t}\exp(-r^2/(4t)).$$
We will use operator splitting and solve these two equations separately:
$$\partial_t c - D \partial_x^2 c = 0$$
$$\partial_t c - D \partial_y^2 c = 0$$

In [ ]:
class Diffusionx:
    
    def __init__(self, X, D, dcdx2):
        pass
        
class Diffusiony:
    
    def __init__(self, X, D, dcdy2):
        pass

In [ ]:
N = 200
grid_x = field.UniformPeriodicGrid(N, 20)
grid_y = field.UniformPeriodicGrid(N, 20)
domain = field.Domain((grid_x, grid_y))
x, y = domain.values()
xm, ym = domain.plotting_arrays()

r = np.sqrt((x-10)**2 + (y-10)**2)

IC = np.exp(-r**2/4)
# t = 2, so evolve forward for 1 unit
target = 1/2*np.exp(-r**2/8)

To timestep this problem, we will use an `ImplicitTimestepper`.

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
pcm = ax.pcolormesh(xm, ym, c.data)
ax.set_aspect(1)
fig.colorbar(pcm)
ax.set_xlabel('x')
ax.set_ylabel('y')
fig.canvas.draw()
while ts_x.t < 1 - 1e-5:
    step_xy(dt)
    if ts_x.iter % 1 == 0:
        pcm.set_array(np.ravel(c.data))
        pcm.set_clim([0,np.max(c.data)])
        fig.canvas.draw()
print(ts_x.t - 1)